# BatchNorm 计算演示
以 [2, 3, 4, 4] 的张量为例，展示 BatchNorm 的计算过程。

In [5]:
import numpy as np

# 固定随机种子，保证结果可复现
np.random.seed(42)

# 生成 [2, 3, 4, 4]，值在 [0, 9] 之间的整数（你给的形状是 2x3x4x4）
x = np.random.randint(0, 10, size=(2, 3, 4, 4))
print("张量形状:", x.shape)
print("张量内容:\n", x)

张量形状: (2, 3, 4, 4)
张量内容:
 [[[[6 3 7 4]
   [6 9 2 6]
   [7 4 3 7]
   [7 2 5 4]]

  [[1 7 5 1]
   [4 0 9 5]
   [8 0 9 2]
   [6 3 8 2]]

  [[4 2 6 4]
   [8 6 1 3]
   [8 1 9 8]
   [9 4 1 3]]]


 [[[6 7 2 0]
   [3 1 7 3]
   [1 5 5 9]
   [3 5 1 9]]

  [[1 9 3 7]
   [6 8 7 4]
   [1 4 7 9]
   [8 8 0 8]]

  [[6 8 7 0]
   [7 7 2 0]
   [7 2 2 0]
   [4 9 6 9]]]]


In [6]:
# ========= BatchNorm 计算（per-channel）=========
# 归一化维度：跨 batch、高和宽，对每个通道分别统计
x = x.astype(np.float32)
N, C, H, W = x.shape
eps = 1e-5

# 每个通道的均值和方差：shape -> (1, C, 1, 1)
mu = x.mean(axis=(0, 2, 3), keepdims=True)
var = x.var(axis=(0, 2, 3), keepdims=True)

print("--- 每个通道的均值 mu_c 和方差 var_c ---")
print("mu:", mu.reshape(-1))
print("var:", var.reshape(-1))


--- 每个通道的均值 mu_c 和方差 var_c ---
mu: [4.65625 5.      4.78125]
var: [6.100586 9.5      9.045898]


In [7]:
# 标准化
x_hat = (x - mu) / np.sqrt(var + eps)

print("--- 标准化后的张量 x_hat ---")
print(x_hat)

--- 标准化后的张量 x_hat ---
[[[[ 0.5440419  -0.6705632   0.94891024 -0.2656949 ]
   [ 0.5440419   1.758647   -1.0754316   0.5440419 ]
   [ 0.94891024 -0.2656949  -0.6705632   0.94891024]
   [ 0.94891024 -1.0754316   0.13917351 -0.2656949 ]]

  [[-1.2977707   0.64888537  0.         -1.2977707 ]
   [-0.32444268 -1.6222134   1.2977707   0.        ]
   [ 0.973328   -1.6222134   1.2977707  -0.973328  ]
   [ 0.32444268 -0.64888537  0.973328   -0.973328  ]]

  [[-0.25975502 -0.92472786  0.40521783 -0.25975502]
   [ 1.0701907   0.40521783 -1.2572143  -0.59224147]
   [ 1.0701907  -1.2572143   1.4026772   1.0701907 ]
   [ 1.4026772  -0.25975502 -1.2572143  -0.59224147]]]


 [[[ 0.5440419   0.94891024 -1.0754316  -1.8851684 ]
   [-0.6705632  -1.4803      0.94891024 -0.6705632 ]
   [-1.4803      0.13917351  0.13917351  1.758647  ]
   [-0.6705632   0.13917351 -1.4803      1.758647  ]]

  [[-1.2977707   1.2977707  -0.64888537  0.64888537]
   [ 0.32444268  0.973328    0.64888537 -0.32444268]
   [-1.2977707

In [ ]:
# 可学习参数（这里演示用 gamma=1, beta=0；真实训练中它们是可学习的）
gamma = np.ones((1, C, 1, 1), dtype=np.float32)
beta = np.zeros((1, C, 1, 1), dtype=np.float32)

# BN 输出
y = gamma * x_hat + beta  # shape: (N, C, H, W)

In [ ]:
# 校验：标准化后的每个通道均值≈0、方差≈1
xhat_mu = x_hat.mean(axis=(0, 2, 3))
xhat_var = x_hat.var(axis=(0, 2, 3))
print("\n--- 归一化后每个通道的均值/方差（应接近 0 和 1） ---")
print("x_hat mean per channel:", xhat_mu)
print("x_hat var  per channel:", xhat_var)

print("\nBN 输出 y 的形状:", y.shape)

## BatchNorm 计算图示（简要）

![BatchNorm 计算图示](./images/BN.png)

# LayerNorm 计算演示
以 [2, 4, 8] 的张量为例，展示 LayerNorm 的计算过程。

In [10]:
import numpy as np

# 固定随机种子，保证结果可复现
np.random.seed(45)

# 生成 [2, 4, 8]，值在 [0, 9] 之间的整数（你给的形状是 2x4x8）
x = np.random.randint(0, 10, size=(2, 4, 8))
print("张量形状:", x.shape)
print("张量内容:\n", x)

张量形状: (2, 4, 8)
张量内容:
 [[[3 0 5 3 4 9 8 1]
  [5 9 6 8 7 8 5 2]
  [8 1 6 4 8 4 6 4]
  [9 1 6 8 8 1 6 0]]

 [[4 9 8 0 9 2 6 7]
  [0 0 2 9 2 6 0 9]
  [6 0 8 8 0 6 7 8]
  [5 1 3 7 5 9 3 2]]]


In [11]:
# ========= LayerNorm 计算（per-token over embedding）=========
# 归一化维度：最后一维（embedding_size=8），对每个 (batch, token) 独立统计
x = x.astype(np.float32)
eps = 1e-5

# 每个 token 的均值与方差：shape -> (2, 4, 1)
mu = x.mean(axis=-1, keepdims=True)
var = x.var(axis=-1, keepdims=True)

print("--- 每个 token 的均值 mu 和方差 var ---")
print("mu:\n", mu)
print("var:\n", var)

--- 每个 token 的均值 mu 和方差 var ---
mu:
 [[[4.125]
  [6.25 ]
  [5.125]
  [4.875]]

 [[5.625]
  [3.5  ]
  [5.375]
  [4.375]]]
var:
 [[[ 8.609375]
  [ 4.4375  ]
  [ 4.859375]
  [11.609375]]

 [[ 9.734375]
  [13.5     ]
  [10.234375]
  [ 6.234375]]]


In [ ]:
# 标准化
x_hat = (x - mu) / np.sqrt(var + eps)

print("--- 标准化后的张量 x_hat ---")
print(x_hat)

In [ ]:
# 可学习参数（演示：gamma=1, beta=0；实际训练中为可学习参数）
# 形状需与被归一化的维度相同，这里是 (1, 1, 8) 以便广播到 (2, 4, 8)
gamma = np.ones((1, 1, x.shape[-1]), dtype=np.float32)
beta = np.zeros((1, 1, x.shape[-1]), dtype=np.float32)

# LN 输出
y = gamma * x_hat + beta  # shape: (2, 4, 8)

print("\n--- 每个 (batch, token) 的均值 mu 和方差 var ---")
print("mu shape:", mu.shape)
print("var shape:", var.shape)
print("mu (前2个token示例):\n", mu.reshape(2, 4)[:,:2])
print("var(前2个token示例):\n", var.reshape(2, 4)[:,:2])

# 校验：归一化后每个 (batch, token) 的均值≈0、方差≈1
xhat_mu = x_hat.mean(axis=-1)  # shape: (2, 4)
xhat_var = x_hat.var(axis=-1)  # shape: (2, 4)
print("\n--- 归一化后每个 token 的均值/方差（应接近 0 和 1） ---")
print("x_hat mean per token:\n", xhat_mu)
print("x_hat var  per token:\n", xhat_var)

print("\nLN 输出 y 的形状:", y.shape)

## LayerNorm 计算图示（简要）
![LayerNorm 计算图示](./images/LN.png)